<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitsky](https://yorko.github.io) (@yorko) 

# <center>Assignment #2. Fall 2019
## <center> Part 2. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

This time we decided to share a pretty decent CatBoost baseline, you'll have to improve the provided solution.

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 
 6. mlcourse.ai tutorials on [categorical feature encoding](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding) (by Wayde Herman) and [CatBoost](https://www.kaggle.com/mitribunskiy/tutorial-catboost-overview) (by Mikhail Tribunskiy)
 7. Last but not the least: [Public Kernels](https://www.kaggle.com/c/flight-delays-fall-2018/notebooks) in this competition

### Your task is to:
 1. beat **"A2 baseline (10 credits)"** on Public LB (**0.75914** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with A1 and the [course rating](https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/) (to appear on 16.09.2019)
 
This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A2: 2019 October 6, 20:59 CET (London time)

In [94]:
import warnings
import numpy as np
import pandas as pd
import catboost

from catboost import cv, Pool
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [2]:
catboost.__version__

'0.17.4'

In [159]:
days_from_start = {
    1: 0, 2: 31, 3: 59, 4: 90, 5: 120, 6: 151, 7: 181,
    8: 212, 9: 243, 10: 273, 11: 304, 12: 334
}

def change_data(data):
    flag = False
    
    for name in "Month DayofMonth DayOfWeek".split():
        data[name] = data[name].str[2:].astype('int')
        
    
    data.DepTime = (data.DepTime % 2400)
    data.DepTime = data.DepTime  // 100 * 60 + data.DepTime % 100
    
    data.Distance = data.Distance.apply(np.log10)
    
    data["days"] = data.Month.map(days_from_start) + data.DayofMonth
    
    data['sin_time'] = np.sin(2 * np.pi * data.DepTime / (24*60))
    data['cos_time'] = np.cos(2 * np.pi * data.DepTime / (24*60))    
    
    data["sin_days"] = np.sin(2 * np.pi * data.days / (365.15))
    data["cos_days"] = np.cos(2 * np.pi * data.days / (365.15))

In [36]:
# data.drop(columns="DepTime", inplace=True)

**Read the data**

In [179]:
train_df = pd.read_csv('../data/flight_delays_train.csv')
test_df = pd.read_csv('../data/flight_delays_test.csv')


In [180]:
train_df['flight'] = train_df['Origin'] + ' --> ' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + ' --> ' + test_df['Dest']

In [181]:
change_data(train_df)
change_data(test_df)
train_df.dep_delayed_15min = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0})

In [172]:
train_df.head()

Month  DayofMonth  DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0      8          21          7     1174            AA    ATL  DFW  2.864511   
1      4          20          3      948            US    PIT  MCO  2.921166   
2      9           2          5      862            XE    RDU  CLE  2.619093   
3     11          25          6      615            OO    DEN  MEM  2.940516   
4     10           7          6     1108            WN    MDW  OMA  2.626340   

   dep_delayed_15min       flight  days  sin_time  cos_time  sin_days  \
0                  0  ATL --> DFW   233 -0.917060  0.398749 -0.762824   
1                  0  PIT --> MCO   110 -0.838671 -0.544639  0.948608   
2                  0  RDU --> CLE   245 -0.580703 -0.814116 -0.879188   
3                  0  DEN --> MEM   329  0.442289 -0.896873 -0.582693   
4                  1  MDW --> OMA   280 -0.992546  0.121869 -0.994429   

   cos_days  
0 -0.646606  
1 -0.316454  
2 -0.476475  
3  0.812693  
4  0.105413

In [127]:
train_df.corr()

Month  DayofMonth  DayOfWeek   DepTime  Distance  \
Month              1.000000    0.007106   0.004717 -0.001187  0.005923   
DayofMonth         0.007106    1.000000  -0.015537  0.000858  0.002541   
DayOfWeek          0.004717   -0.015537   1.000000  0.009001  0.010622   
DepTime           -0.001187    0.000858   0.009001  1.000000 -0.032721   
Distance           0.005923    0.002541   0.010622 -0.032721  1.000000   
dep_delayed_15min  0.016998    0.025842   0.010439  0.241020  0.013264   
days               0.996431    0.091234   0.003409 -0.001132  0.006104   
sin_time           0.000546    0.000462  -0.010936 -0.891212  0.037040   
cos_time          -0.004452    0.001733  -0.004571  0.566127 -0.012694   
sin_days          -0.783774   -0.013147   0.001035  0.001024 -0.000672   
cos_days           0.007678   -0.004549   0.001880 -0.003474 -0.002135   

                   dep_delayed_15min      days  sin_time  cos_time  sin_days  \
Month                       0.016998  0.996431  0.000546 -0.004452 -0.783774   
DayofMonth                  0.025842  0.091234  0.000462  0.001733 -0.013147   
DayOfWeek                   0.010439  0.003409 -0.010936 -0.004571  0.001035   
DepTime                     0.241020 -0.001132 -0.891212  0.566127  0.001024   
Distance                    0.013264  0.006104  0.037040 -0.012694 -0.000672   
dep_delayed_15min           1.000000  0.019030 -0.216927  0.166761 -0.009746   
days                        0.019030  1.000000  0.000591 -0.004331 -0.783404   
sin_time                   -0.216927  0.000591  1.000000 -0.324402 -0.000451   
cos_time                    0.166761 -0.004331 -0.324402  1.000000  0.004405   
sin_days                   -0.009746 -0.783404 -0.000451  0.004405  1.000000   
cos_days                   -0.005791  0.011356  0.001299 -0.007173  0.000175   

                   cos_days  
Month              0.007678  
DayofMonth        -0.004549  
DayOfWeek          0.001880  
DepTime           -0.003474  
Distance          -0.002135  
dep_delayed_15min -0.005791  
days               0.011356  
sin_time           0.001299  
cos_time          -0.007173  
sin_days           0.000175  
cos_days           1.000000

In [128]:
test_df.head()

Month  DayofMonth  DayOfWeek  DepTime UniqueCarrier Origin Dest  Distance  \
0      7          25          3      375            YV    MRY  PHX  2.776701   
1      4          17          2      459            WN    LAS  HOU  3.091667   
2     12           2          7      411            MQ    GSP  ORD  2.761176   
3      3          25          7      974            WN    BWI  MHT  2.576341   
4      6           6          3      905            UA    ORD  STL  2.411620   

   days  sin_time  cos_time  sin_days  cos_days  
0   206  0.997859 -0.065403 -0.392251 -0.919858  
1   107  0.908143 -0.418660  0.963673 -0.267085  
2   336  0.975342 -0.220697 -0.480819  0.876820  
3    84 -0.894934 -0.446198  0.992148  0.125069  
4   157 -0.722364 -0.691513  0.426005 -0.904721

---
---
---

In [182]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx = sorted(np.append(categ_feat_idx, [0, 1, 2]))
print(categ_feat_idx)

[0, 1, 2, 4, 5, 6, 8]


In [183]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].values
X_test = test_df.values

In [184]:
X_train_part, X_valid, \
y_train_part, y_valid = train_test_split(X_train, y_train,
                                         test_size=0.3,
                                         random_state=17,
                                         shuffle=True
                                         )
                                        

In [185]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

#### Train

In [166]:
%%time
ctb.fit(X_train_part,
        y_train_part,
        cat_features=categ_feat_idx);

Wall time: 1min 33s


In [167]:
ctb_valid_pred = ctb.predict_proba(X_train_part)[:, 1]
print(roc_auc_score(y_train_part, ctb_valid_pred))

ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, ctb_valid_pred))

0.8461853389948719
0.754556922886649


K Fold

In [186]:
test_score = []
train_score = []
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

i = 0
for test_index, train_index in skf.split(X_train, y_train):
    
    X_train_1, X_test_1 = X_train[train_index, :], X_train[test_index, :]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]
    
    
    _ctb = CatBoostClassifier(random_seed=17,
                              silent=True
                             ).fit(X_train_1, y_train_1, cat_features=categ_feat_idx)
    
    test_score.append(roc_auc_score(y_test_1,
                                    _ctb.predict_proba(X_test_1)[:, 1]
                                   )
                     )
    train_score.append(roc_auc_score(y_train_1, _ctb.predict_proba(X_train_1)[:, 1]))
    
    i += 1
    print(i)
    

1
2
3


In [187]:
print(np.mean(test_score))
print(test_score)
print()
print(np.mean(train_score))
print(train_score)

0.7432098583982043
[0.7463960861707694, 0.7437989748619579, 0.7394345141618857]

0.8143489015028306
[0.815838258639185, 0.8093454332198605, 0.8178630126494463]


params = \
{
# "iterations": 100,
"eval_metric": "AUC",
"verbose": False
#"devices": -1
}

pool = Pool(data=X_train,
            label=y_train,
            cat_features=categ_feat_idx)

scores = cv(pool,
            params=params,
            fold_count=5,
            plot="True",
            early_stopping_rounds=50)

----
----
----

Result

In [97]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx);

Wall time: 2min 26s


In [98]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [99]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv('../data/sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = ctb_test_pred
    sample_sub.to_csv('ctb_pred_4.csv')

In [123]:
!head ctb_pred.csv

id,dep_delayed_15min
0,0.029813427154851885
1,0.045432589326845726
2,0.07735792880819185
3,0.26298352270959774
4,0.23812453664806577
5,0.1015030549268571
6,0.040331757718181545
7,0.2408800371330324
8,0.11304498960469873


In [127]:
!head ctb_pred_3.csv

id,dep_delayed_15min
0,0.025572166172376693
1,0.045981976846112715
2,0.07101399403966373
3,0.2905628227350732
4,0.2716783001872599
5,0.08292537325175686
6,0.04404007262086365
7,0.19666702216697723
8,0.07454022433182766


In [100]:
!head ctb_pred_4.csv

id,dep_delayed_15min
0,0.041335027204167984
1,0.06414436097806048
2,0.05168336229185614
3,0.31269384580379495
4,0.3241744296848872
5,0.09560777045922966
6,0.04826738267201204
7,0.21209648956603291
8,0.0913154450303372
